<a href="https://colab.research.google.com/github/smaboudian/recurrence-analysis/blob/main/interactive_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive recurrence plots

Open this notebook in Google Colab to view interactive features:
https://colab.research.google.com/drive/11uSkmbwWUZnGNztV1ulUWQxZ2VnhWnwX?usp=sharing

In [5]:
import pandas as pd
import numpy as np

import plotly 
import plotly.express as px
from plotly.offline import iplot
import plotly.graph_objects as go


# Plotting functions

In [6]:
# basic dot plot functions 

def delta(x,y):
    return 1 if x == y and x!= 'nan' and y!='nan' else 0

def M(seq1,seq2,i,j,k):
    return sum(delta(x,y) for x,y in zip(seq1[i:i+k],seq2[j:j+k]))

def makeMatrix(seq1,seq2,k):
    n = len(seq1)
    m = len(seq2)
    return [[M(seq1,seq2,i,j,k) for j in range(m-k+1)] for i in range(n-k+1)]

In [7]:
def ind_dotplot_plotly(seqx, diag_x, listnum_x, 
                             seqy, diag_y, listnum_y, 
                             valid_items_list):

    noblanks_x = [i for i in seqx if i!='nan']
    noblanks_y = [i for i in seqy if i!='nan']
    new_dotplot = np.array(makeMatrix(noblanks_y,noblanks_x,1))
    fig = go.Figure()

    #dotplot: 
    fig.add_trace(go.Heatmap(
                    z=new_dotplot,
                    name="dotplot",
                    colorscale = 'Blues',
                    showscale=False))
    
    x_labels = noblanks_x
    y_labels = noblanks_y
    

    # to plot missing on axis: 
    
    paired_seqs = [noblanks_x, noblanks_y]
    seqx_p,seqy_p=paired_seqs
    in1_not2 = list(set(seqx_p)-set(seqy_p))
    in2_not1 = list(set(seqy_p)-set(seqx_p))
    
    x_pts =[]
    enum_x = enumerate(noblanks_x)
    enum_x_list = list(enum_x)
    for j in range(len(enum_x_list)):
        for missing in in1_not2:
            if enum_x_list[j][1]==missing:
                x_pts += [enum_x_list[j][0]]
    x_axis_points = x_pts
    
    fig.add_trace(
        go.Scatter(
            x=x_axis_points,
            y=[-1 for i in range(len(x_axis_points))],
            name = 'in L1, not L2',
        mode='markers',
            marker=dict(
                color='Orange',
                symbol = 'line-ns',
                line=dict(width=5,
                    color='Orange',
                ))))
    y_pts =[]
    enum_y = enumerate(noblanks_y)
    enum_y_list = list(enum_y)
    for k in range(len(enum_y_list)):
        for missing in in2_not1:
            if enum_y_list[k][1]==missing:
                y_pts += [enum_y_list[k][0]]
    y_axis_points = y_pts
    
    fig.add_trace(
        go.Scatter(
            x=[-1 for i in range(len(y_axis_points))],
            y=y_axis_points,
            name = 'in L2, not L1',
        mode='markers',
            marker=dict(
                color='Orange',
                symbol = 'line-ew',
                line=dict(width=5,
                    color='Orange',
                ))))
   
    
    # marking repeats:
    
    x_tick_repeats = [i for i, x in enumerate(noblanks_x) if noblanks_x.count(x) > 1]
    y_tick_repeats = [i for i, x in enumerate(noblanks_y) if noblanks_y.count(x) > 1]
    x_coords_reps = []
    y_coords_reps = []
    if x_tick_repeats != []:
        for idx in x_tick_repeats:
            if noblanks_x[idx] in noblanks_y:
                x_coords_reps += [idx]
                y_coords_reps_list = [i for i, y in enumerate(noblanks_y) if y==noblanks_x[idx]]
                y_coords_reps += [y_coords_reps_list[0]]
                
                y_coords_reps_list = y_coords_reps_list[1:]
                while y_coords_reps_list!=[]:
                    x_coords_reps += [idx]
                    y_coords_reps += [y_coords_reps_list[0]]
                    y_coords_reps_list = y_coords_reps_list[1:]
            else:
                x_coords_reps += [idx]
                y_coords_reps += [-1]
    if y_tick_repeats != []:
        for idx in y_tick_repeats:
            if noblanks_y[idx] in noblanks_x:
                y_coords_reps += [idx]
                x_coords_reps_list = [i for i, x in enumerate(noblanks_x) if x==noblanks_y[idx]]
                x_coords_reps += [x_coords_reps_list[0]]
                
                x_coords_reps_list = x_coords_reps_list[1:]
                while x_coords_reps_list!=[]:
                    y_coords_reps += [idx]
                    x_coords_reps += [x_coords_reps_list[0]]
                    x_coords_reps_list = x_coords_reps_list[1:]
            else:
                y_coords_reps += [idx]
                x_coords_reps += [-1]
    fig.add_trace(
        go.Scatter(
            x=x_coords_reps,
            y=y_coords_reps,
            name = 'repeated item',
        mode='markers',
            marker=dict(
                color='lightgreen',
                symbol = 'triangle-right',
                line=dict(
                    color='Black',
                    width=.5
                ))))
                    
    
    #showing invalid entries (intrusions):
    
    def find_intrusions(list_1):
        valid_list_lowercase = [str.lower(item) for item in valid_items_list]
        not_in_valid_list = []
        for item_j in list_1:
            if item_j not in valid_list_lowercase:
                not_in_valid_list += [item_j]
        return not_in_valid_list
     
    not_in_list = find_intrusions(noblanks_x) + find_intrusions(noblanks_y)
    
    invalids = not_in_list
    
    x_tick_invalids = [i for i, x in enumerate(noblanks_x) if x in invalids]
    y_tick_invalids = [i for i, x in enumerate(noblanks_y) if x in invalids]
    
    if x_tick_invalids != []:
        for idx in x_tick_invalids:
            x_labels[idx]=x_labels[idx]+'*'
        fig.add_trace(
            go.Scatter(
                x=x_tick_invalids,
                y=[-1 for i in range(len(x_tick_invalids))],
                name = 'invalid item',
            mode='markers',
                marker=dict(
                    color='Red',
                    symbol = 'x',
                    line=dict(width=0.5,
                        color='Black',
                    ))))
            
    if y_tick_invalids != []:
        for idx in y_tick_invalids:
            y_labels[idx]=y_labels[idx]+'*'
        fig.add_trace(
            go.Scatter(
                x=[-1 for i in range(len(y_tick_invalids))],
                y=y_tick_invalids,
                name = 'invalid item',
            mode='markers',
                marker=dict(
                    color='Red',
                    symbol = 'x',
                    line=dict(width=0.5,
                        color='Black',
                    ))))

    # figure layout:
    fig.update_layout(
            width=500,
            height=500,
            xaxis=go.layout.XAxis(
                title=go.layout.xaxis.Title(
                    text='L1: {0}, List {1}'.format(diag_x,listnum_x),
                    font=dict(
                        family="Helvetica",
                        size=18,
                        color="#7f7f7f"
                    ))),
            yaxis=go.layout.YAxis(
                title=go.layout.yaxis.Title(
                    text='L2: {0}, List {1}'.format(diag_y,listnum_y),
                    font=dict(
                        family="Helvetica",
                        size=18,
                        color="#7f7f7f"
                    )))
        )

    fig.update_xaxes(showline=True, linewidth=1, linecolor='grey', 
                     tickangle=270,
                    ticktext=x_labels,
                     tickvals = [i for i in range(len(x_labels))],
                    ticks="inside",
                     showgrid=True, 
                     gridcolor='LightGrey')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='grey',
                    ticktext=y_labels,
                     tickvals = [i for i in range(len(y_labels))],
                    showgrid=True, 
                     gridcolor='LightGrey')

    fig.update_yaxes(autorange="reversed")
    fig.update_layout(legend=dict(x=1, y=1))

    fig['layout']['xaxis']['side'] = 'top'

    fig.show()


# Import data 
(1) fluency data: csv of fluency lists to plot  
(2) list of valid words: text file containing list of words allowed for given fluency category

## Fluency data (`data`)

Sample fluency data provided was downloaded from https://osf.io/j6qea/ (Zemla & Austerweil, 2019), and then was cleaned to remove typos.

(To upload your own data, uncomment cells below.)

In [8]:
# sample data:
url = 'https://raw.githubusercontent.com/smaboudian/recurrence-analysis/main/ucsd_fluency_cleaned.csv'

data = pd.read_csv(url, 
    converters={'item': eval})
data.head()

,id,listnum,group,SEX,EDUC,DRS,MMSE,category,rank,item
0,5,4,NC,2,16,144,30,animals,1.0,"[armadillo, horse, cow, goat, pig, rabbit, mou..."
1,6,4,NC,1,19,144,30,animals,1.0,"[mouse, rat, raccoon, dog, cat, elephant, cow,..."
2,93,1,NC,2,14,143,30,animals,1.0,"[dog, cat, ape, elephant, monkey, chimpanzee, ..."
3,93,2,NC,2,14,138,26,animals,1.0,"[owl, rhinoceros, buffalo, horse, dog, cat, el..."
4,93,3,NC,2,14,143,30,animals,1.0,"[dog, rat, camel, horse, cat, ferret, hamster,..."


In [9]:
# to import your own local data, uncomment and run the lines below (in this cell & the following one):
#from google.colab import files
#uploaded = files.upload()

In [10]:
# to import your own data, uncomment and run the lines below after running the cell above:
#import io
#filename = 'your_file.csv' #change this line
#data = pd.read_csv(io.BytesIO(uploaded[filename]))
#data.head()

## List of valid words (`valid_items`)

List of words allowed for each category, used to indicate when a word is an out-of-category intrusion (list provided is for "animals" category)


In [11]:
url = 'https://raw.githubusercontent.com/smaboudian/recurrence-analysis/main/animals_list.txt'
valid_items = list(np.loadtxt(url, dtype=np.str, delimiter='\n'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning:

`np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



# Example plots 


In [12]:
# example plot 1: manually inputted data (not from csv file)

list1 = ['camel', 'elephant', 'giraffe', 'monkey', 'gorilla', 'seal', 'parakeet', 'cockatiel', 'cockatoo', 'crow', 'hawk', 'donkey', 'lion', 'tiger', 'stork', 'duck', 'goose', 'panda']
list2 = ['lion', 'tiger', 'monkey', 'elephant','ball','leash', 'dog', 'cat', 'camel', 'snake', 'mouse', 'dog', 'monkey', 'giraffe', 'fish', 'cow', 'horse', 'sheep', 'goat']

group1 = '[group]' #group info or diagnosis, e.g. MCI or AD
group2 = '[group]'
list1num = '#1, MM/DD/YY' #x-axis list number and/or date
list2num = '#2, MM/DD/YY' #y-axis list number and/or date
valid_items_list = valid_items


ind_dotplot_plotly(list1, group1, list1num, 
                         list2, group2, list2num, 
                         valid_items_list)


In [23]:
# example plot 2, from imported csv file

row1 = data.iloc[3] #row of the dataset with the 1st list to plot
row2 = data.iloc[4] #row of the dataset with the 2nd list to plot

list1 = row1['item']
list2 = row2['item']

group1 = str(row1['group']) #group info or diagnosis, e.g. MCI or AD
group2 = str(row2['group'])
list1num = str(row1['listnum']) #x-axis list number and/or date
list2num = str(row2['listnum']) #y-axis list number and/or date
valid_items_list = valid_items


ind_dotplot_plotly(list1, group1, list1num, 
                         list2, group2, list2num, 
                         valid_items_list)


In [19]:

# example plot 3, from imported csv file

row1 = data.iloc[1164]
row2 = data.iloc[1165]

list1 = row1['item']
list2 = row2['item']

group1 = str(row1['group']) #group info or diagnosis, e.g. MCI or AD
group2 = str(row2['group'])
list1num = str(row1['listnum']) #x-axis list number and/or date
list2num = str(row2['listnum']) #y-axis list number and/or date
valid_items_list = valid_items

ind_dotplot_plotly(list1, group1, list1num, 
                         list2, group2, list2num, 
                         valid_items_list)


In [15]:
# example plot 4, from imported csv file

row1 = data.iloc[1147]
row2 = data.iloc[1148]

list1 = row1['item']
list2 = row2['item']

group1 = str(row1['group']) #group info or diagnosis, e.g. MCI or AD
group2 = str(row2['group'])
list1num = str(row1['listnum']) #x-axis list number and/or date
list2num = str(row2['listnum']) #y-axis list number and/or date
valid_items_list = valid_items

ind_dotplot_plotly(list1, group1, list1num, 
                         list2, group2, list2num, 
                         valid_items_list)
